# Automated Query Performance Insights in Snowflake Notebooks

In this notebook, we'll provide SQL queries that you can use to analyze query history and gain insights on performance and bottlenecks.

The following 6 queries against the `ACCOUNT_USAGE` schema provide insight into the past performance of queries (examples 1-4), warehouses (example 5), and tasks (example 6).

## 1. Top n longest-running queries

This query provides a listing of the top n (50 in the example below) longest-running queries in the last day. You can adjust the `DATEADD` function to focus on a shorter or longer period of time. Replace `STREAMLIT_DEMO_APPS` with the name of a warehouse.

In [ ]:
SELECT query_id,
  ROW_NUMBER() OVER(ORDER BY partitions_scanned DESC) AS query_id_int,
  query_text,
  total_elapsed_time/1000 AS query_execution_time_seconds,
  partitions_scanned,
  partitions_total,
FROM snowflake.account_usage.query_history Q
WHERE warehouse_name = 'STREAMLIT_DEMO_APPS' AND TO_DATE(Q.start_time) > DATEADD(day,-1,TO_DATE(CURRENT_TIMESTAMP()))
  AND total_elapsed_time > 0 --only get queries that actually used compute
  AND error_code IS NULL
  AND partitions_scanned IS NOT NULL
ORDER BY total_elapsed_time desc
LIMIT 50;

## 2. Queries organized by execution time over past month

This query groups queries for a given warehouse by buckets for execution time over the last month. These trends in query completion time can help inform decisions to resize warehouses or separate out some queries to another warehouse. Replace `STREAMLIT_DEMO_APPS` with the name of a warehouse.

In [ ]:
SELECT
  CASE
    WHEN Q.total_elapsed_time <= 1000 THEN 'Less than 1 second'
    WHEN Q.total_elapsed_time <= 60000 THEN '1 second to 1 minute'
    WHEN Q.total_elapsed_time <= 300000 THEN '1 minute to 5 minutes'
    ELSE 'more than 5 minutes'
  END AS BUCKETS,
  COUNT(query_id) AS number_of_queries
FROM snowflake.account_usage.query_history Q
WHERE  TO_DATE(Q.START_TIME) >  DATEADD(month,-1,TO_DATE(CURRENT_TIMESTAMP()))
  AND total_elapsed_time > 0
  AND warehouse_name = 'STREAMLIT_DEMO_APPS'
GROUP BY 1;

## 3. Find long running repeated queries

You can use the query hash (the value of the query_hash column in the ACCOUNT_USAGE QUERY_HISTORY view) to find patterns in query performance that might not be obvious. For example, although a query might not be excessively expensive during any single execution, a frequently repeated query could lead to high costs, based on the number of times the query runs.

You can use the query hash to identify the queries that you should focus on optimizing first. For example, the following query uses the value in the query_hash column to identify the query IDs for the 100 longest-running queries:

In [ ]:
SELECT
    query_hash,
    COUNT(*),
    SUM(total_elapsed_time),
    ANY_VALUE(query_id)
  FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
  WHERE warehouse_name = 'STREAMLIT_DEMO_APPS'
    AND DATE_TRUNC('day', start_time) >= CURRENT_DATE() - 7
  GROUP BY query_hash
  ORDER BY SUM(total_elapsed_time) DESC
  LIMIT 100;

## 4. Track the average performance of a query over time

The following statement computes the daily average total elapsed time for all queries that have a specific parameterized query hash (7f5c370a5cddc67060f266b8673a347b).

In [ ]:
SELECT
    DATE_TRUNC('day', start_time),
    SUM(total_elapsed_time),
    ANY_VALUE(query_id)
  FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
  WHERE query_parameterized_hash = '7f5c370a5cddc67060f266b8673a347b'
    AND DATE_TRUNC('day', start_time) >= CURRENT_DATE() - 30
  GROUP BY DATE_TRUNC('day', start_time);

## 5. Total warehouse load
This query provides insight into the total load of a warehouse for executed and queued queries. These load values represent the ratio of the total execution time (in seconds) of all queries in a specific state in an interval by the total time (in seconds) for that interval.

For example, if 276 seconds was the total time for 4 queries in a 5 minute (300 second) interval, then the query load value is 276 / 300 = 0.92.

In [ ]:
SELECT TO_DATE(start_time) AS date,
  warehouse_name,
  SUM(avg_running) AS sum_running,
  SUM(avg_queued_load) AS sum_queued
FROM snowflake.account_usage.warehouse_load_history
WHERE TO_DATE(start_time) >= DATEADD(month,-1,CURRENT_TIMESTAMP())
GROUP BY 1,2
HAVING SUM(avg_queued_load) >0;

## 6. Longest running tasks
This query lists the longest running tasks in the last day, which can indicate an opportunity to optimize the SQL being executed by the task.

In [ ]:
SELECT DATEDIFF(seconds, query_start_time,completed_time) AS duration_seconds,*
FROM snowflake.account_usage.task_history
WHERE state = 'SUCCEEDED'
  AND query_start_time >= DATEADD (week, -1, CURRENT_TIMESTAMP())
ORDER BY duration_seconds DESC;

## Resources

Queries used in this notebook is from the [Snowflake Docs](https://docs.snowflake.com/) on [Exploring execution times](https://docs.snowflake.com/en/user-guide/performance-query-exploring)